# Preparación del dataset para el análisis por GSVA y Bitfam

### *Sheila Santomé* 

Esta libreta permite generar los archivos de salida requeridos para el análisis de GSVA y Bitfam en R, a partir de objetos AnnData. En el presente caso, el procedimiento se aplica al objeto combinado (merge) correspondiente a los ratones. No obstante, el flujo de trabajo es generalizable a cualquier otro objeto AnnData, modificando las rutas de entrada y salida según sea necesario.

-------

## Importar paquetes

In [ ]:
import anndata
import mygene
import numpy as np
import pandas as pd
import pickle
import scanpy as sc
import scvi
from scvi.external import SOLO
from scvi.model import SCVI
from scipy.io import mmread
import seaborn as sns


# Datos GSVA

## Cargar datos necesarios

Los archivos AnnData se han almacenado en formato pickle, lo que permite un manejo más eficiente y reduce el tamaño de almacenamiento. En este bloque de código, las rutas deberán modificarse según los archivos de entrada correspondientes. Es importante que los archivos pickle contengan los datos no normalizados, ya que el flujo de trabajo en R requiere esta versión de los datos para su correcto procesamiento.

In [ ]:
f1_path = "./pickles/counts_mouse_3_barcode_qcmetrics.pkl"
f2_path = "./pickles/counts_mouse_1_barcode_qcmetrics.pkl"

with open(f1_path, "rb") as f1, \
    open(f2_path, "rb") as f2:
    
    counts_a = pickle.load(f1)
    counts_b = pickle.load(f2)
  

### Unificar los datos de los dos ratones

In [ ]:

counts_a.obs["Sample"] = "Mouse_1"
counts_b.obs["Sample"] = "Mouse_3"

adata = counts_a.concatenate(counts_b, batch_key="Sample", batch_categories=["Mouse_1", "Mouse_3"])

# --- Verificar ---
adata.obs["Sample"].value_counts()

C:\Users\Sheimoon\AppData\Local\Temp\ipykernel_3272\1734178596.py:6: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata = counts_mouse_1.concatenate(counts_mouse_3, batch_key="Sample", batch_categories=["Mouse_1", "Mouse_3"])


Sample
Mouse_3    16499
Mouse_1     5195
Name: count, dtype: int64

In [10]:
adata.var

,mt,ribo,n_cells-Mouse_1,n_cells_by_counts-Mouse_1,mean_counts-Mouse_1,pct_dropout_by_counts-Mouse_1,total_counts-Mouse_1,n_cells-Mouse_3,n_cells_by_counts-Mouse_3,mean_counts-Mouse_3,pct_dropout_by_counts-Mouse_3,total_counts-Mouse_3
Xkr4,False,False,25,25,0.002102,99.789810,25,57,57,0.003088,99.691174,57
Mrpl15,False,True,2241,2241,0.356903,81.158567,4245,5106,5106,0.413177,72.335699,7626
Lypla1,False,False,1523,1523,0.165630,87.195224,1970,2414,2414,0.149374,86.920951,2757
Tcea1,False,False,4131,4131,0.776274,65.268202,9233,7118,7118,0.609633,61.434686,11252
Rgs20,False,False,2023,2023,0.285186,82.991424,3392,2656,2656,0.187083,85.609796,3453
...,...,...,...,...,...,...,...,...,...,...,...,...
Csprs,False,False,39,39,0.003447,99.672104,41,32,32,0.001788,99.826624,33
Vamp7,False,False,1662,1662,0.195309,86.026568,2323,2491,2491,0.155659,86.503766,2873
Tmlhe,False,False,546,546,0.051202,95.409450,609,382,382,0.021672,97.930325,400
CAAA01147332.1,False,False,441,441,0.040188,96.292248,478,483,483,0.026873,97.383107,496


## Seleccionar los genes altamente variables

Para evitar que el análisis por GSVA requiera un tiempo de procesado extenso, se selccionaran los 5000 genes más altamente variables.

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor='cell_ranger',        
    n_top_genes=5000        
)

hvg_genes = adata.var.index[adata.var['highly_variable']].tolist()

In [13]:
adata= adata[:, hvg_genes].copy()

adata.var

,mt,ribo,n_cells-Mouse_1,n_cells_by_counts-Mouse_1,mean_counts-Mouse_1,pct_dropout_by_counts-Mouse_1,total_counts-Mouse_1,n_cells-Mouse_3,n_cells_by_counts-Mouse_3,mean_counts-Mouse_3,pct_dropout_by_counts-Mouse_3,total_counts-Mouse_3,highly_variable,means,dispersions,dispersions_norm
Rgs20,False,False,2023,2023,0.285186,82.991424,3392,2656,2656,0.187083,85.609796,3453,True,0.220568,1.681077,3.557826
Atp6v1h,False,False,3022,3022,0.436439,74.592231,5191,4302,4302,0.303570,76.691770,5603,True,0.363234,1.543182,1.159650
Rb1cc1,False,False,4005,4005,0.655288,66.327560,7794,4243,4243,0.292355,77.011432,5396,True,0.419425,1.720560,1.034490
St18,False,False,579,579,0.074659,95.131999,888,297,297,0.017771,98.390854,328,True,0.034480,2.088611,16.747101
Rrs1,False,False,1803,1803,0.248865,84.841096,2960,4141,4141,0.302974,77.564068,5592,True,0.275606,1.388448,0.779377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd5,True,False,10946,10946,8.021103,7.970405,95403,14872,14872,2.499323,19.423525,46130,True,3.408223,5.024587,0.882829
mt-Nd6,True,False,5228,5228,0.843703,56.045065,10035,2860,2860,0.186488,84.504524,3442,True,0.323546,2.078744,5.335099
mt-Cytb,True,False,11645,11645,69.541197,2.093493,827123,18350,18350,34.622745,0.579726,639032,True,37.304139,29.141183,14.794270
Tmlhe,False,False,546,546,0.051202,95.409450,609,382,382,0.021672,97.930325,400,True,0.032682,1.139444,1.095938


## Pasar de objeto Anndata a matriz de expresión

En los siguientes bloques de código se realiza la conversión de un objeto AnnData a una matriz de expresión génica, formato requerido para el análisis mediante GSVA. Además, se lleva a cabo la conversión de la nomenclatura génica al formato de identificadores de Ensembl (Ensembl IDs), descargado previamente en la notebook de **Análisis Funcional** , necesaria para la compatibilidad con las bases de datos de anotaciones utilizadas en el entorno de R.

In [ ]:
adata_df = pd.DataFrame(
    adata.X.T.toarray() if hasattr(adata.X, "toarray") else adata.X.T,
    index=adata.var_names,   # genes
    columns=adata.obs_names  # muestras
)

In [ ]:

# Resetear el índice para que los nombres de genes sean una columna
adata_df = adata_df.reset_index().rename(columns={'index': 'Gene name'})

# Cargar el archivo con el id de los genes
genes = pd.read_csv("./Gen_markers/gen_id.csv", sep=",")

merged = pd.merge(
    adata_df,
    genes,
    on="Gene name",
    how="left"
)

# Eliminar la columna 'Gene name' 
merged = merged.drop(columns=["Gene name"])

In [ ]:
merged = merged.dropna(subset=['Gene stable ID'])

merged

,mouse_1_AAACCCAAGATACCAA-1-Mouse_1,mouse_1_AAACCCAAGTGTTGAA-1-Mouse_1,mouse_1_AAACCCACACAATTCG-1-Mouse_1,mouse_1_AAACCCACACTCCACT-1-Mouse_1,mouse_1_AAACCCACATCAGTGT-1-Mouse_1,mouse_1_AAACCCATCAATCTCT-1-Mouse_1,mouse_1_AAACCCATCACTTGGA-1-Mouse_1,mouse_1_AAACCCATCATCGCAA-1-Mouse_1,mouse_1_AAACCCATCTCGACGG-1-Mouse_1,mouse_1_AAACGAACAAGGCTTT-1-Mouse_1,...,mouse_3_TTTGTTGGTCTTACTT-1-Mouse_3,mouse_3_TTTGTTGGTGACTCGC-1-Mouse_3,mouse_3_TTTGTTGGTGCATTAC-1-Mouse_3,mouse_3_TTTGTTGGTGCGTCGT-1-Mouse_3,mouse_3_TTTGTTGGTGTCTCCT-1-Mouse_3,mouse_3_TTTGTTGTCCACTTTA-1-Mouse_3,mouse_3_TTTGTTGTCCATCTGC-1-Mouse_3,mouse_3_TTTGTTGTCCGAGTGC-1-Mouse_3,mouse_3_TTTGTTGTCTCGAGTA-1-Mouse_3,Gene stable ID
0,0,0,0,0,2,0,0,1,0,0,...,0,0,0,0,3,0,0,0,0,ENSMUSG00000002459
1,0,1,2,0,1,0,0,3,0,1,...,0,2,0,0,0,1,2,0,2,ENSMUSG00000033793
2,0,2,0,1,5,4,0,1,1,5,...,0,0,1,0,0,0,0,0,0,ENSMUSG00000025907
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSMUSG00000033740
4,0,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,2,0,0,ENSMUSG00000061024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5007,3,12,5,0,9,42,0,21,12,22,...,2,6,6,5,6,7,6,4,2,ENSMUSG00000064360
5008,12,31,5,0,27,70,2,36,44,55,...,7,13,14,18,15,11,19,10,8,ENSMUSG00000064363
5009,4,18,5,0,7,24,1,11,8,14,...,1,5,4,0,1,3,1,1,3,ENSMUSG00000064367
5010,0,3,1,0,4,7,1,5,2,0,...,0,1,0,0,0,0,0,0,0,ENSMUSG00000064368


In [ ]:
#Hacer que la columna de Ensembl IDs sea la primera
cols = ['Gene stable ID'] + [col for col in merged.columns if col != 'Gene stable ID']

merged_clean = merged[cols]

print(merged_clean.head())


       Gene stable ID  mouse_1_AAACCCAAGATACCAA-1-Mouse_1  \
0  ENSMUSG00000002459                                   0   
1  ENSMUSG00000033793                                   0   
2  ENSMUSG00000025907                                   0   
3  ENSMUSG00000033740                                   0   
4  ENSMUSG00000061024                                   0   

   mouse_1_AAACCCAAGTGTTGAA-1-Mouse_1  mouse_1_AAACCCACACAATTCG-1-Mouse_1  \
0                                   0                                   0   
1                                   1                                   2   
2                                   2                                   0   
3                                   0                                   0   
4                                   1                                   0   

   mouse_1_AAACCCACACTCCACT-1-Mouse_1  mouse_1_AAACCCACATCAGTGT-1-Mouse_1  \
0                                   0                                   2   
1               

## Guardar archivos

Los archivos de salida generados serán un archivo Feather para los counts y un archivo CSV para los metadatos. El formato Feather resulta más eficiente que CSV, ya que ocupa menos espacio en disco y permite una lectura y escritura significativamente más rápida, optimizando así el procesamiento posterior en R. Las variables *out_counts* y *out_metadata* se modificarán de aciuerdo al nombre que se le quiera dar a los archivos. 

In [ ]:
out_counts="file_name.feather"

merged.to_feather(out_counts)


In [ ]:
metadata = adata.obs

out_metadata="file_name.csv"

metadata.to_csv(out_metadata)


-------------
# Procesar archivos cuando es necesaria información de los datos tras normalizar y clusterizar

En algunos casos, se requiere realizar el análisis por GSVA después de generar un UMAP. Para ello, es necesario cotejar la información entre el objeto AnnData clusterizado (posterior a la normalización y reducción de dimensiones) y el objeto AnnData sin normalizar. Esta correspondencia garantiza que las asignaciones de celdas y metadatos se mantengan consistentes a lo largo del flujo de análisis.

## Cargar archivos
Va a ser necesario cargar el archivo previo a la normalización y el archivo obtenido tras el UMAP.

In [11]:
f1_path = "./pickles/counts_pool_before_barcode_qc_metrics.pkl"
f2_path  = "./pickles/counts_pool_before_leiden.pkl"

with open(f1_path, 'rb') as f1, \
     open(f2_path, "rb") as f2 :

    counts = pickle.load(f1)
    counts_normalized = pickle.load(f2)   

## Sacar valores de clústeres de UMAP
En este caso lo que se busca es añadir la columna con el clúster al que pertenece cada célula tras realizar el UMAP, estos datos están almacenados en la columna "leiden" del objeto Anndata normalizado.

In [ ]:
# Extraer clusters Leiden de las células normalizadas
leiden_series = counts_normalized.obs["leiden"]

# Añadir columna Leiden al obs del otro AnnData
counts.obs["leiden"] = leiden_series.reindex(counts.obs_names)

In [13]:
# Checkear que no haya valores NA
na_count = counts.obs['leiden'].isna().sum()
print(f"Valores NA en 'leiden': {na_count}")

Valores NA en 'leiden': 0


En caso de haber valores NA será necesario ejecutar el siguiente bloque de código

In [ ]:
counts = counts[~counts.obs['leiden'].isna(), :]

# Seleccionar Genes altamente variables
A partir de aquí, el proceso será el mismo que en el caso anterior.

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor='cell_ranger',        
    n_top_genes=5000        
)

hvg_genes = adata.var.index[adata.var['highly_variable']].tolist()


In [ ]:
adata= adata[:, hvg_genes].copy()

adata.var

## Pasar de objeto Anndata a matriz de expresión


In [ ]:
adata_df = pd.DataFrame(
    adata.X.T.toarray() if hasattr(adata.X, "toarray") else adata.X.T,
    index=adata.var_names,   # genes
    columns=adata.obs_names  # muestras
)

In [ ]:

# Resetear el índice para que los nombres de genes sean una columna
adata_df = adata_df.reset_index().rename(columns={'index': 'Gene name'})

# Cargar el archivo con el id de los genes
genes = pd.read_csv("./Gen_markers/gen_id.csv", sep=",")

merged = pd.merge(
    adata_df,
    genes,
    on="Gene name",
    how="left"
)

# Eliminar la columna 'Gene name'
merged = merged.drop(columns=["Gene name"])

In [ ]:
merged = merged.dropna(subset=['Gene stable ID'])

merged

In [ ]:
#Hacer que la columna de Ensembl IDs sea la primera
cols = ['Gene stable ID'] + [col for col in merged.columns if col != 'Gene stable ID']

merged_clean = merged[cols]

print(merged_clean.head())

## Guardar archivos
Las variables *out_counts* y *out_metadata* se modificarán de aciuerdo al nombre que se le quiera dar a los archivos. 

In [ ]:
out_counts="file_name.feather"

merged.to_feather(out_counts)


In [ ]:
metadata = adata.obs

out_metadata="file_name.csv"

metadata.to_csv(out_metadata)